In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score
import lightgbm as lgb
from collections import Counter
import warnings
import pickle
from catboost import CatBoostClassifier
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.model_selection import train_test_split
import pickle
from sklearn import tree
from sklearn import tree
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,classification_report,f1_score

class AdaBoostClassifier:
    '''A simple AdaBoost Classifier.'''

    def __init__(self, n_weakers_limit):
        self.weights = []
        self.classifiers = []
        self.n_weakers_limit = n_weakers_limit
        self.ws = []
        pass

    def is_good_enough(self,X_train,y_train):
        #if((len(self.classifiers) > self.n_weakers_limit) or
        #        accuracy_score(y_train, self.predict(X_train))>=0.9):
        if(len(self.classifiers) > self.n_weakers_limit):
            C_report = classification_report(y_train, self.predict(X_train), digits=4, output_dict=True)
            print(classification_report(y_train, self.predict(X_train), digits=4))
            report = f1_score(y_train, self.predict(X_train), average=None)
            print('Score: ', report)
            df = pd.DataFrame(C_report).transpose()
            df.to_csv("classifier_report.csv", index= True)
            return True
        else:
            # print("准确率：" + str(accuracy_score(y_train, self.predict(X_train))))
            print(classification_report(y_train, self.predict(X_train), digits=4))
            report = f1_score(y_train, self.predict(X_train), average=None)
            print('Score: ', report)
            return False
        pass

    def fit(self,X_train,y_train):
        w = np.ones((1, len(X_train))) / len(y_train)
        flag = 0
        while flag == 0 or (not (self.is_good_enough(X_train, y_train))):
            # for i in range(2):
            # 定义并训练基分类器决策树
            flag = 1
            clf = tree.DecisionTreeClassifier \
                (class_weight="balanced", criterion='gini', max_depth=1, splitter="random", random_state=0)
            predit_tree = clf.fit(X_train, y_train, w.reshape(-1))
            # predit_tree = clf.fit(X_train, y_train)
            self.classifiers.append(predit_tree)
            # 计算错误率
            correct_rate = predit_tree.score(X_train, y_train)
            error_rate = 1 - correct_rate
            # 计算分类器权重
            alpha = 1 / 2 * np.log((1 - error_rate) / error_rate)
            # 存储分类器权重
            self.weights.append(alpha)
            # 计算预测结果
            result = predit_tree.predict(X_train).reshape(-1, 1)
            # 更新数据集权重参数
            first_part = -alpha * y_train
            second_part = np.exp(first_part * result)
            third_part = (w.reshape(-1, 1) * second_part)
            z_t = np.sum(third_part)
            w = w.reshape(-1, 1) * second_part / z_t
            self.ws.append(w)
            print("alpha=", alpha)
        pass


    def predict_scores(self, X_train):
        '''Calculate the weighted sum score of the whole base classifiers for given samples.

        Args:
            X: An ndarray indicating the samples to be predicted, which shape should be (n_samples,n_features).

        Returns:
            An one-dimension ndarray indicating the scores of differnt samples, which shape should be (n_samples,1).
        '''
        scores = np.zeros((1, len(X_train)))
        for i in range(len(self.classifiers)):
            scores +=  self.classifiers[i].predict(X_train)*self.weights[i]    
            
        return scores
        pass

    def predict(self, X_train, threshold=0):
        '''Predict the catagories for geven samples.

        Args:
            X: An ndarray indicating the samples to be predicted, which shape should be (n_samples,n_features).
            threshold: The demarcation number of deviding the samples into two parts.

        Returns:
            An ndarray consists of predicted labels, which shape should be (n_samples,1).
        '''
        scores = self.predict_scores(X_train)
        y_train = np.where(scores>threshold, 1, -1)
        return y_train.reshape(-1,1)
        pass

    @staticmethod
    def save(model, filename):
        with open(filename, "wb") as f:
            pickle.dump(model, f)

    @staticmethod
    def load(filename):
        with open(filename, "rb") as f:
            return pickle.load(f)

In [ ]:
train = pd.read_csv('is_train_data/is_train_20190701.txt')
train = pd.merge(train[['link','current_slice_id','future_slice_id']][(train.label == 2)],train,how='left') 


In [ ]:
for i in range(2, 10):
    train1 = pd.read_csv('is_train_data/is_train_2019070'+str(i)+'.txt')
    train = train.append(pd.merge(train1[['link','current_slice_id','future_slice_id']][(train1.label == 2)],train1,how='left'))


In [ ]:
train

In [ ]:
y = np.where(y == 2 , 1, -1)

In [ ]:
AdaBoost = AdaBoostClassifier(20)

In [ ]:
from sklearn.model_selection import train_test_split
# X_train, X_validation, y_train, y_validation = train_test_split(train.loc[:,'time_diff':'width'],y,test_size=0.25 , random_state=1234)
X_train, X_validation, y_train, y_validation = train_test_split(train,y,test_size=0.2 , random_state=1234)

In [ ]:
AdaBoost.fit(X_train, y_train.reshape(-1,1))